In [20]:
# import the libraries
import numpy as np
import pandas as pd 
from active_learning import adsp_r as AL
from sklearn.model_selection import train_test_split

# data = pd.read_excel("Concrete_Data.xls")
data = np.loadtxt("concrete_data.txt")
# print(data.head(5))
x_data = data[:,:-1]
y_data = data[:,-1]

In [18]:
print(x_data.shape)

(1030, 8)


In [19]:
# split 
Xtrain, Xtest, ytrain, ytest = train_test_split(x_data, y_data,train_size=0.80)

In [21]:
whole_data = {"x_data":Xtrain, "y_data":ytrain}

model = AL.active_learn(whole_data,50,0.05,1000)

x,y = model.get_more_samples()

print(x.shape)

np.savetxt("optimal_x.csv",x,delimiter=",")
np.savetxt("optimal_y.csv",y,delimiter=",")

np.savetxt("x_validation.csv",model.x_test,delimiter=",")
np.savetxt("y_validation.csv",model.y_test,delimiter=",")

[0.042111389830670226, 0.029951905123537282, 0.010439911503028905, 0.0012504062066235873]
[0.1386180024486887, 0.036147681750725524, 0.11283454394047403, 0.13262869654538145, 0.0249625540054434, 0.019980222032407305, 0.2613541769785794, 0.4766229066026979, 0.13187089573443891, 0.130061688435797, 0.03686913483141896, 0.14087216431491165, 0.10882020487435375, 0.15769379005046846, 0.12939348088718497, 0.08704416028298544, 0.259902565117513, 0.11674283300387248, 0.8330521121373125, 5.060370884312238, 0.06023553121637744, 0.17536483555022556, 0.03804457891122339, 0.24519169164101115, 0.27895771068768793, 0.09654356648542452, 0.1705860160070572, 0.17874391832990302, 0.07191245136932069, 0.741491110220966, 2.5622262622302094, 0.07589924803109717, 0.09743560903871322, 0.32727373588436837, 0.06217551929598526, 0.034020387208020544, 0.16429990640457165, 0.2976637733956405, 0.5140993023884398, 0.01629031514529551, 0.06564254522003467, 0.4119241788620497, 0.7580185345230066, 0.13362821259780458, 0

In [22]:
x_train = pd.read_csv("optimal_x.csv",header=None)
y_train = pd.read_csv("optimal_y.csv",header=None)
# x_test = pd.read_csv("x_test.csv",header=None)
# y_test = pd.read_csv("y_test.csv",header=None)

In [23]:
from smt.surrogate_models import KRG as SMT_KRG
from sklearn.metrics import r2_score
smt_model = SMT_KRG(corr='abs_exp')
smt_model.set_training_values(np.array(x_train),np.array(y_train))
smt_model.train()
smt_model_y_test = smt_model.predict_values(np.array(Xtest))
R2 = r2_score(ytest,smt_model_y_test)
print(R2)

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 461
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  4.4746888
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 206
   
   Predicting ...
   Predicting - done. Time (sec):  0.0509894
   
   Prediction time/pt. (sec) :  0.0002475
   
0.873019237269598


In [37]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.preprocessing import LabelEncoder

# kernel = RBF()
# Define the anisotropic kernel
length_scale = [[0.1]]*8
# kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=length_scale, length_scale_bounds=(1e-3, 1e3))
kernel = RBF(length_scale=length_scale, length_scale_bounds=(1e-3, 1e3))
# Define the Gaussian process regressor model
model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5)
# Train the Gaussian process regressor model
model.fit(x_train, y_train)
print(model.kernel_.theta)
print(model.log_marginal_likelihood())

c:\Users\Kenny\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


[ 5.0712325   5.38031155  5.08942035  3.99137945  0.42288626  2.27266687
  4.64220276 -2.30258509]
-170638274274.3526


In [36]:
y_pred, std = model.predict(Xtest, return_std=True)
R2 = r2_score(ytest,y_pred)
print(R2)

0.3591533198011583


In [127]:
# squar_exp matern52

dataset = np.loadtxt("concrete_data.txt")
x_data = dataset[:,:-1]
y_data = dataset[:,-1] 
loop = 20
R2_set = []
for l in range(loop):
    Xtrain, Xtest, ytrain, ytest = train_test_split(x_data, y_data,train_size=149)
    smt_model_1 = SMT_KRG(corr='abs_exp')
    smt_model_1.set_training_values(np.array(Xtrain),np.array(ytrain))
    smt_model_1.train()
    # smt_model_1_y_test = smt_model_1.predict_values(np.array(Xtest))
    # R2 = r2_score(ytest,smt_model_1_y_test)
    smt_model_1_y_test = smt_model_1.predict_values(np.array(x_data))
    R2 = r2_score(y_data,smt_model_1_y_test)
    R2_set.append(R2)
print(R2_set)
print("Mean R2: {}".format(np.average(R2_set)))

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 149
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.0877581
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1030
   
   Predicting ...
   Predicting - done. Time (sec):  0.0149992
   
   Prediction time/pt. (sec) :  0.0000146
   
___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 149
   
___________________________________________________________________________
   
 T

___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1030
   
   Predicting ...
   Predicting - done. Time (sec):  0.0255921
   
   Prediction time/pt. (sec) :  0.0000248
   
___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 149
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.0723298
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1030
   
   Predicting ...
   Predicting - done. Time (sec):  0.0156221
   
   Prediction time/pt. (sec) :  0.0000152
   
___________________________________________________________________________
   
                       

In [12]:
x_train = pd.read_csv("optimal_x.csv",header=None)
y_train = pd.read_csv("optimal_y.csv",header=None)
x_test = pd.read_csv("x_test.csv",header=None)
y_test = pd.read_csv("y_test.csv",header=None)

In [16]:
from smt.surrogate_models import KRG as SMT_KRG
from sklearn.metrics import r2_score
smt_model = SMT_KRG(corr='abs_exp')
smt_model.set_training_values(np.array(x_train),np.array(y_train))
smt_model.train()
smt_model_y_test = smt_model.predict_values(np.array(x_test))
R2 = r2_score(y_test,smt_model_y_test)
print(R2)

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 348
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  2.0129910
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 682
   
   Predicting ...
   Predicting - done. Time (sec):  0.1152625
   
   Prediction time/pt. (sec) :  0.0001690
   
0.5964349816682425


In [15]:
# squar_exp matern52
from sklearn.model_selection import train_test_split
dataset = np.loadtxt("concrete_data.txt")
x_data = dataset[:,:-1]
y_data = dataset[:,-1] 
loop = 5
R2_set = []
for l in range(loop):
    Xtrain, Xtest, ytrain, ytest = train_test_split(x_data, y_data,train_size=0.30)
    smt_model_1 = SMT_KRG(corr='abs_exp')
    smt_model_1.set_training_values(np.array(Xtrain),np.array(ytrain))
    smt_model_1.train()
    # smt_model_1_y_test = smt_model_1.predict_values(np.array(Xtest))
    # R2 = r2_score(ytest,smt_model_1_y_test)
    smt_model_1_y_test = smt_model_1.predict_values(np.array(Xtest))
    R2 = r2_score(ytest,smt_model_1_y_test)
    R2_set.append(R2)
print('\n')
print('*'*80)
print('\n')
print(R2_set)
print('\n')
print('*'*80)
print('\n')
print("Mean R2: {}".format(np.average(R2_set)))

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 309
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  1.8293521
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 721
   
   Predicting ...
   Predicting - done. Time (sec):  0.0886793
   
   Prediction time/pt. (sec) :  0.0001230
   
___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 309
   
___________________________________________________________________________
   
 Tr